## Data preparation and cleaning



In [74]:
import sqlite3
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import  mean_squared_error, r2_score
import warnings
#Connect to database
conn = sqlite3.connect('cleaned_customer_support.db')
data = pd.read_sql_query("SELECT * FROM cleaned_customer_support", conn)
y = data['CSAT_Score']

In [75]:
#First going to evaluate the hour when the issue was reported as well as the response time 
X1 = data[['issue_reported_hour_of_day']]
X2 = data[['response_time_minutes']]
#First model based on issue_reported_hour_of_day
X1_train, X1_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, random_state=42)
model1 = LinearRegression()
model1.fit(X1_train, y_train)
y_pred1 = model1.predict(X1_test)
rmse1 = np.sqrt(mean_squared_error(y_test, y_pred1))
r2_1 = r2_score(y_test, y_pred1)
#Printing relevant data for issue_reported_hour_of_day
print("Model using 'issue_reported_hour_of_day':")
print(f'RMSE: {rmse1:.4f}')
print(f'R-squared: {r2_1:.4f}')
examples = X1_test.copy()  
examples['Actual_CSAT_Score'] = y_test 
examples['Predicted_CSAT_Score'] = y_pred1  
print(examples.head(10))  
# Second model based on response_time_minutes
X2_train, X2_test, y_train, y_test = train_test_split(X2, y, test_size=0.2, random_state=42)
model2 = LinearRegression()
model2.fit(X2_train, y_train)
y_pred2 = model2.predict(X2_test)
rmse2 = np.sqrt(mean_squared_error(y_test, y_pred2))
r2_2 = r2_score(y_test, y_pred2)
#Printing relevant data for response_time_minutes
print("\nModel using 'response_time_minutes':")
print(f'RMSE: {rmse2:.4f}')
print(f'R-squared: {r2_2:.4f}')
examples = X2_test.copy()  
examples['Actual_CSAT_Score'] = y_test 
examples['Predicted_CSAT_Score'] = y_pred2  
print(examples.head(10))  

Model using 'issue_reported_hour_of_day':
RMSE: 1.5386
R-squared: -0.0010
       issue_reported_hour_of_day  Actual_CSAT_Score  Predicted_CSAT_Score
3394                           13                  4              4.062751
23665                          19                  5              4.071607
8895                           21                  1              4.074559
13218                           9                  4              4.056847
12382                          13                  5              4.062751
2932                            0                  1              4.043563
20001                          18                  5              4.070131
26043                          21                  5              4.074559
3023                            7                  5              4.053895
5102                           13                  5              4.062751

Model using 'response_time_minutes':
RMSE: 1.5218
R-squared: 0.0207
       response_time_minutes  Ac

In [76]:
# Third model based on Agent Shift which needs one hot encoder to preprocess the data since the shifts are split into known ones
encoder = OneHotEncoder() 
X3_encoded = encoder.fit_transform(data[['Agent Shift']])
shift_categories = encoder.categories_[0] 

# Train model
X3_train, X3_test, y_train, y_test = train_test_split(X3_encoded, y, test_size=0.2, random_state=42)
model3 = LinearRegression()
model3.fit(X3_train, y_train)
y_pred3 = model3.predict(X3_test)
mse3 = mean_squared_error(y_test, y_pred3)
r2_3 = r2_score(y_test, y_pred3)
mapped_categories = {int(i): category[0] for i, category in enumerate(shift_categories)}

# Convert test data to DataFrame and map encoded categories to their labels
X3_test_df = pd.DataFrame.sparse.from_spmatrix(X3_test)
X3_test_df = X3_test_df.rename(columns=mapped_categories)
X3_test_df = X3_test_df.astype(int)

X3_test_df.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)
examples = X3_test_df.copy()  
examples['Actual_CSAT_Score'] = y_test 
examples['Predicted_CSAT_Score'] = y_pred3 
rmse3 = np.sqrt(mean_squared_error(y_test, y_pred3))
r2_2 = r2_score(y_test, y_pred3)
print("\nModel using 'Agent Shift':")
print(shift_categories)
print(f'RMSE: {rmse3:.4f}')
print(f'R-squared: {r2_2:.4f}')
print(examples.head(10))





Model using 'Agent Shift':
['Afternoon' 'Evening' 'Morning' 'Night' 'Split']
RMSE: 1.5361
R-squared: 0.0023
   A  E  M  N  S  Actual_CSAT_Score  Predicted_CSAT_Score
0  0  0  1  0  0                  4              4.001691
1  0  0  1  0  0                  5              4.001691
2  1  0  0  0  0                  1              4.137886
3  0  0  1  0  0                  4              4.001691
4  0  1  0  0  0                  5              4.096031
5  0  1  0  0  0                  1              4.096031
6  0  1  0  0  0                  5              4.096031
7  0  1  0  0  0                  5              4.096031
8  0  0  0  0  1                  5              4.338692
9  0  0  1  0  0                  5              4.001691


In [77]:
#Here we will run through all the columns and try to know which one creates the best linear regression model
rmse_dict = {}
for column in data.columns:
    X = None
    if column in ['Unique id','Customer_Remarks','Issue_reported at','issue_responded','Survey_response_Date']:
        continue
    elif column in ['channel_name', 'category', 'Sub-category','Tenure Bucket','Agent Shift']:
          # Create categories using OneHotEncoder
        encoder = OneHotEncoder(drop='first')
        X_encoded = encoder.fit_transform(data[[column]])
        X = pd.DataFrame.sparse.from_spmatrix(X_encoded)
    elif column in ['Agent_name', 'Supervisor', 'Manager']:
        # Here we apply get dummies to put numerical values for this feature where there are a lot of different people
        X = pd.get_dummies(data[column], prefix=column)
    else:
        # Use the column as is
        X = data[[column]]

    model = LinearRegression()
    model.fit(X, y)
    
    y_pred = model.predict(X)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    rmse_dict[column] = rmse
    

for column, rmse in rmse_dict.items():
    print(f'{column}, RMSE: {rmse:.4f}')

# Print top 3 RMSE values with their corresponding columns
top_3_rmse = sorted(rmse_dict.items(), key=lambda x: x[1])[:3]
print("\nTop 3 RMSE values when applying linear regression on one column:")
for column, rmse in top_3_rmse:
    print(f'{column}, RMSE: {rmse:.4f}')


channel_name, RMSE: 1.5578
category, RMSE: 1.5495
Sub-category, RMSE: 1.5193
Agent_name, RMSE: 1.4712
Supervisor, RMSE: 1.5506
Manager, RMSE: 1.5552
Tenure Bucket, RMSE: 1.5570
Agent Shift, RMSE: 1.5580
CSAT_Score, RMSE: 0.0000
response_time_minutes, RMSE: 1.5383
issue_reported_hour_of_day, RMSE: 1.5600
issue_reported_day_of_week, RMSE: 1.5600

Top 3 RMSE values when applying linear regression on one column:
CSAT_Score, RMSE: 0.0000
Agent_name, RMSE: 1.4712
Sub-category, RMSE: 1.5193


In [86]:
warnings.filterwarnings("ignore")# Just removing the warning for sparse arrays
def preprocess_data(data):
    processed_data = pd.DataFrame()  
    for column in data.columns:
        if column in ['Unique id','Customer_Remarks','Issue_reported at','issue_responded','Survey_response_Date','CSAT_Score']:
            continue
        elif column in ['channel_name', 'category', 'Sub-category','Tenure Bucket','Agent Shift']:
            encoder = OneHotEncoder(drop='first')
            X_encoded = encoder.fit_transform(data[[column]])
            X = pd.DataFrame.sparse.from_spmatrix(X_encoded)
        elif column in ['Agent_name', 'Supervisor', 'Manager']:
            X = pd.get_dummies(data[column], prefix=column)
        else:
            # Use the column as is
            X = data[[column]]
        processed_data = pd.concat([processed_data, X], axis=1) 
    processed_data.columns = processed_data.columns.astype(str)  
    return processed_data

def evaluate_model(X_train, X_test, y_train, y_test):
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return rmse

def random_feature_combinations(features, num_combinations, max_columns=5):
    combinations = []
    for _ in range(num_combinations):
        shuffled_features = list(features)  
        np.random.shuffle(shuffled_features)
        num_columns = np.random.randint(1, min(max_columns, len(shuffled_features)) + 1)
        combination = tuple(shuffled_features[:num_columns])
        combinations.append(combination)
    return combinations





processed_data = preprocess_data(data)
X_train, X_test, y_train, y_test = train_test_split(processed_data, y, test_size=0.2, random_state=42)
print(data.columns)
features = processed_data.columns
print(features)
num_combinations = 1000
max_columns = 10
combinations = random_feature_combinations(features, num_combinations, max_columns)
results = {}
for i, combination in enumerate(combinations):
    print(f"Evaluating combination {i+1}/{num_combinations}: {combination}")
    X_train_comb = X_train[list(combination)]
    X_test_comb = X_test[list(combination)]
    rmse = evaluate_model(X_train_comb, X_test_comb, y_train, y_test)
    results[combination] = rmse

sorted_results = sorted(results.items(), key=lambda x: x[1])
print("\nTop 5 Best Combinations:")
for i, (combination, rmse) in enumerate(sorted_results[:5], 1): 
    print(f"{i}. Combination: {combination}, RMSE: {rmse:.4f}")


Index(['Unique id', 'channel_name', 'category', 'Sub-category',
       'Customer_Remarks', 'Issue_reported at', 'issue_responded',
       'Survey_response_Date', 'Agent_name', 'Supervisor', 'Manager',
       'Tenure Bucket', 'Agent Shift', 'CSAT_Score', 'response_time_minutes',
       'issue_reported_hour_of_day', 'issue_reported_day_of_week'],
      dtype='object')
Index(['0', '1', '0', '1', '2', '3', '4', '5', '6', '7',
       ...
       '1', '2', '3', '0', '1', '2', '3', 'response_time_minutes',
       'issue_reported_hour_of_day', 'issue_reported_day_of_week'],
      dtype='object', length=1496)
Evaluating combination 1/1000: ('Agent_name_Karen Williams',)
Evaluating combination 2/1000: ('Agent_name_Chloe Roberts', 'Agent_name_Nancy Baker', 'Agent_name_Brandy Baker', 'Agent_name_Taylor Harvey', 'Agent_name_Michael Abbott', 'Agent_name_Jessica Allen', 'Agent_name_James Tran')
Evaluating combination 3/1000: ('Agent_name_Shelley Watson', 'Agent_name_Linda Butler', 'Agent_name_Amy Mend